### Script de modelagem direta da bacia de barreirinhas - dados grav adquiridos por Nelson Delimar

In [ ]:
# importando pacotes necessarios:
import numpy as np # pacote numerico
import pylab as py 
import matplotlib.pyplot as plt # pacote de plotagem
from matplotlib import widgets
from matplotlib.path import Path
from matplotlib.patches import PathPatch
%matplotlib tk

### Importando os pacotes com as nossas funcoes:

In [ ]:
from modules.iplots import pick_points, model_masses # para fazer o plot interativo
from modules.gravity import grav2D_anom, g_sphere # funcao que calcula dados de gravidade (rao 1994)

### Leitura dos dados reais do perfil de Barreirinhas:

In [ ]:
# leitura no script do pc do bijani:
elev, gz, x_utm, y_utm = np.loadtxt('../dados_reais/Barreirinhas/barreirinhas_perfil.txt', 
                                    skiprows=1 , usecols = (2,3,0,1), unpack = True)
#alterando o sinal de elev devido a mudança da referencia do eixo z (positivo para baixo):
elev = -elev                                   

In [ ]:
# Plot dos dados para verificação:
plt.figure(figsize=(15,7), facecolor='w')
plt.scatter(x_utm,gz,s=5.0,c=gz,marker='*', cmap=plt.cm.coolwarm)
#plt.plot(x_utm, gz,'*r')
#plt.title ('Gravity Data (mGal)', fontsize = 16)
plt.xlabel ('UTM x(m)', fontsize = 14)
plt.ylabel ('Bouguer Anomaly (mGal)', fontsize = 14)
plt.grid()
#plt.savefig('Bouguer.png', dpi=300, transparent=True, bbox_inches="tight" )
plt.show()

### Definição da area de plot do modelo:

In [ ]:
area= []
xmin = np.min(x_utm)
xmax = np.max(x_utm)
zmin = np.min( (elev) ) #-100.0
#zmax = 3.0 #profundidade da bacia em km (entradas do rao et al., 1994 em Km)
zmax = 2500.0 #adaptado para metros !
print(zmin)
area = [xmin, xmax, zmin, zmax]
print(area)

### Criacao do modelo de bacia atraves dos clicks na area de plot: 

In [ ]:
%matplotlib tk 
axes = plt.figure().add_subplot(1,1,1)
xv,zv = pick_points(area, axes, marker='o', color='k', size=8, xy2ne=False)

### calculo da anomalia de gravidade produzido pelo modelo de bacia (artigo do Rao et. al., 1994):

In [ ]:
# info da propriedade fisica:
delta_rho = -1.10 #(g/cm³)# contraste de Densidade do arenito em relacao ao embasamento local no topo da bacia!
beta = 7.12
# convertendo para SI(kg/m³):
rho_ref = 2.88 # densidade do embasamento (checar!)
rho = delta_rho + rho_ref
#rho = rho * 1000.0 
print('densidade no topo da bacia em g/cm³=', rho)

In [ ]:
#Criacao de variaveis necessarias para o calculo da anomalia:
nv = np.size (xv)
#print nv
nper = np.size (x_utm)
#print nper
# calculo da anomalia grav atraves de rao1994:
xd = np.zeros( (nv,) )
zd = np.zeros( (nv,) )
grav = np.zeros( (nper,) )
for i in range(nper):
    for j in range(nv):
        xd[j] = ( xv[j] - x_utm[i])/1000.0
        zd[j] = (zv[j]- elev[i])/1000.0
    # chamando a funcao do rao 1994:
    grav[i] = grav2D_anom(xd,zd,delta_rho,beta)

In [ ]:
#######################################################
# computing hyperbolic density function
########################################################
z = np.linspace( np.amin(zv), np.amax(zv), np.size(zv) ) # em km
rhoz = np.zeros( np.size(z) )
rhoz = (delta_rho)*beta**2/(beta + (z/100.0) )**2 # conversao para g/cm3 (CORRETO!!!!)
#rhoz = rhoz + 2.670

In [ ]:
# save figure and a file for reproduction:
A = np.zeros( (len(z),2) )
A[:,0] = z
A[:,1] = rhoz
caminho = 'figs/density1'
# save text file for ploting issues:
np.savetxt(caminho+'.txt', A, header=' Z , rhoZ', delimiter=' ' )

In [ ]:
%matplotlib inline
# plot of the density distribution:
fig = plt.figure(figsize=(7,7))

# definition of fontsize:
fs = 15 
# invert axis
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

# set labelsize 
plt.tick_params(axis='y', labelsize=fs-2)
plt.tick_params(axis='x', labelsize=fs-2,labelbottom=True,labeltop=True)

plt.text(rhoz[0]+0.01, z[0]+0.148, str( format(rhoz[0],'.3f') ), style='italic',
        bbox={'facecolor':'red', 'alpha':0.5, 'pad':10})

plt.plot(rhoz,z,'k-',linewidth=2.0)
plt.plot(rhoz[0],z[0],'ro',linewidth=2.0)
plt.grid()
plt.xlabel(r'Density contrast $(g/cm^3)$',fontsize=fs)
plt.ylabel(r'Depth $(km)$',fontsize=fs)
plt.axis([np.max(rhoz)+0.05, np.min(rhoz)-0.02, np.max(z)+0.1, np.min(z)-0.1])
plt.savefig(caminho +'.png', dpi=300, transparent=True, bbox_inches="tight" )
plt.show()
#############################################################################################################

In [ ]:
# get the number of elements of xv (number of vertices of the polygon)
n = np.size(xv)     
# create new working arrays for the vertices of a closed polygon:
x = np.zeros( (n+1,) )  
z = np.zeros( (n+1,) ) 
x[0:n] = xv
z[0:n] = zv
# GAMBIARRA PARA PLOT DE CORES ASSSOCIADOOS À BACIA MODELADA (PENSAR EM ALGO MILHÓ) 
x[n:n+1] = min(xv)
z[n:n+1] = np.min(zv)

### SALVANDO O ARQUIVO E A RESPECTIVA IMAGEM COM O MESMO NOME

In [ ]:
# save fig and a txt file for reproduction:
caminho = "figs/model_basin_barreirinhas1"

B = np.zeros( (len(z),2) )
B[:,0] = x
B[:,1] = z

# save text file for ploting issues:
np.savetxt(caminho+'.txt', B, header=' UTMx(m) , depth(m)', delimiter=' ' )

In [ ]:
print(np.array([x,z]).T)

In [ ]:
# and the modeled basin:
fig = plt.figure(figsize=(14,8))
path = Path(np.array([x,z]).T)
patch = PathPatch(path, facecolor='none')
#######################################################
plt.gca().add_patch(patch)
plt.plot(xv,zv,'k-o')

# plot the last and the first corner to close up the polygon:
fs = 18 # font size for the label
plt.gca().invert_yaxis()
plt.xlabel(r'UTM x $(m)$',fontsize=fs)
plt.ylabel(r'Depth $(m)$',fontsize=fs)
plt.xlim([np.min(xv), np.max(xv)])

##################################################################
# plot the density variation together with the basin:
##################################################################

im = plt.imshow(rhoz.reshape(np.size(zv),1),  cmap=plt.cm.Wistia,interpolation="bicubic",
                origin='lower',extent=[min(x), max(x), min(z), max(z)],aspect="auto", clip_path=patch, clip_on=True)
#im.set_clip_path(patch)

# OBS:::::: para reverter a escala de cores, basta um simples "_r" ! sucesso de vida!!!!
plt.gca().invert_yaxis()
cbar = plt.colorbar()
cbar.ax.set_ylabel(r'Density constrast in $g/cm^3$', fontsize=fs)

# Mat, verificar!!!!!!
plt.savefig(caminho +".png", dpi=300, transparent=True, bbox_inches='tight')
plt.show()

### Plot do modelo atual + os dados reais para comparação:

In [ ]:
# save fig and a txt file for reproduction:
caminho = "figs/datafit1"
C = np.zeros( (len(x_utm),3) )
C[:,0] = x_utm
C[:,1] = grav
C[:,2] = gz
# save text file for ploting issues:
np.savetxt(caminho+'.txt', C, header=' UTMx(m) Gpred(mGal) Gobs(mGal)', delimiter=' ' )

In [ ]:
fig = plt.figure(figsize=(14,8))

plt.plot(x_utm,gz,'.k', label='observed data')
plt.plot(x_utm,grav,'.r', label='predicted data')
plt.legend()
plt.grid()
#plt.title ('Gravity Data (mGal)', fontsize = 16)
plt.xlabel ('UTM x(m)', fontsize = 14)
plt.ylabel ('Bouguer Anomaly (mGal)', fontsize = 14)
plt.savefig(caminho+'png', dpi=300)
plt.show()

# Etapa nova! Criação da anomalia bouguer produzida por uma intrusão modelada

In [ ]:
# limites para a densidade da intrusao (g/cm3)
arearho= [1.1,1.2,1.1,1.2]

In [ ]:
%matplotlib tk
# obtencao das coordenadas 2D e a densidade da intrusao atraves de clicks:
xb, zb, rhob = model_masses(area, arearho, background=[xv,zv])

In [ ]:
# Calcular a anomalia produzida pelas bolinhas:
intruso=[] # lista para guardar todas as bolinhas:
for i in range(len(xb)):
    intruso.append( [ xb[i],zb[i],rhob[i] ] )
gravb = g_sphere(x_utm, elev, intruso, component='z')

In [ ]:
# principio da superposicao para computar o efeito da poligono (grav) + intrusao (gravb}):
gravt = grav + gravb

In [ ]:
%matplotlib inline
# and the modeled basin:
fig = plt.figure(figsize=(14,8))
path = Path(np.array([x,z]).T)
patch = PathPatch(path, facecolor='none')

#######################################################
plt.gca().add_patch(patch)
plt.plot(xv,zv,'k-o')

# plot the last and the first corner to close up the polygon:
fs = 18 # font size for the label
plt.gca().invert_yaxis()
plt.xlabel(r'UTM x $(m)$',fontsize=fs)
plt.ylabel(r'Depth $(m)$',fontsize=fs)
plt.xlim([np.min(xv), np.max(xv)])

##################################################################
# plot the density variation together with the basin:
##################################################################

im = plt.imshow(rhoz.reshape(np.size(zv),1),  cmap=plt.cm.Wistia, interpolation="bicubic",
                origin='lower',extent=[min(x), max(x), min(z), max(z)],aspect="auto", clip_path=patch, clip_on=True)
#im.set_clip_path(patch)

# OBS:::::: para reverter a escala de cores, basta um simples "_r" ! sucesso de vida!!!!
plt.gca().invert_yaxis()
cbar = plt.colorbar()
cbar.ax.set_ylabel(r'Density constrast in $g/cm^3$', fontsize=fs)


rb = np.array(rhob)-rho_ref
print (rb)
plt.scatter(xb,zb,s=60,c=rb,cmap='Wistia', vmin=min(rhoz), vmax=max(rhoz) )

# Mat, verificar!!!!!!
plt.savefig(caminho +".png", dpi=300, transparent=True, bbox_inches='tight')
plt.show()

### PLOT DO AJUSTE DOS DADOS COM A INTRUSAO:


In [ ]:
# save fig and a txt file for reproduction:
caminho = "figs/datafit_intruso1"
C = np.zeros( (len(x_utm),3) )
C[:,0] = x_utm
C[:,1] = gravt
C[:,2] = gz
# save text file for ploting issues:
np.savetxt(caminho+'.txt', C, header=' UTMx(m) Gpred(mGal) Gobs(mGal)', delimiter=' ' )

In [ ]:
fig = plt.figure(figsize=(14,8))

plt.plot(x_utm,gz,'.k', label='observed data')
plt.plot(x_utm,gravt,'.r', label='predicted data')
plt.legend()
plt.grid()
#plt.title ('Gravity Data (mGal)', fontsize = 16)
plt.xlabel ('UTM x(m)', fontsize = 14)
plt.ylabel ('Bouguer Anomaly (mGal)', fontsize = 14)
plt.savefig(caminho+'png', dpi=300)
plt.show()